# DGD+SCC算法+极小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust1_help=[]
tau_robust1=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
W_robust1=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))


#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust1=[]
neighbor_byzantine_num_robust1=[]
for i in range(0,94):
    h_num_robust1=0
    b_num_robust1=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            b_num_robust1=b_num_robust1+1
        else:
            h_num_robust1=h_num_robust1+1
    neighbor_honest_num_robust1.append(h_num_robust1)
    neighbor_byzantine_num_robust1.append(b_num_robust1)

In [4]:
#DGD+SCC算法 
def Robust_DGD_SCC_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,tau_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+极小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust1*100)):
            c_clipping_dual_robust1=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust1=0#存放clipping后的对偶值和
            #计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust1[i]==0:
                tau_robust1[i][k]=10000
            else:
                weight_sum_robust1=0
                for j in range(0,15):
                    if neighbor_robust1[i][j]!=94 and neighbor_robust1[i][j]!=95 and neighbor_robust1[i][j]!=96 and neighbor_robust1[i][j]!=97 and neighbor_robust1[i][j]!=98 and neighbor_robust1[i][j]!=99:
                        weight_sum_robust1=weight_sum_robust1+(0.0625)*(dual_variable_intermediate_robust1[neighbor_robust1[i][j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i][j]]-dual_variable_intermediate_robust1[i])
                tau_robust1[i][k]=sqrt(weight_sum_robust1/(0.0625*neighbor_byzantine_num_robust1[i]))                       
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])==0:
                    c_clipping_dual_robust1.append(0)
                elif 1<=(tau_robust1[i][k]/(math.sqrt((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])))):
                    c_clipping_dual_robust1.append(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])
                else:
                    c_clipping_dual_robust1.append((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*((tau_robust1[i][k]/(math.sqrt((dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i])*(dual_variable_intermediate_robust1[neighbor_robust1[i,j]]-dual_variable_intermediate_robust1[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust1=mix_sum_c_dual_robust1+0.0625*(dual_variable_intermediate_robust1[i]+c_clipping_dual_robust1[j])
            #诚实用户基于SCC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust1[i][k]=0.0625*dual_variable_intermediate_robust1[i]+mix_sum_c_dual_robust1
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%3000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_SCC_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,tau_robust1)

****************************DGD算法+SCC+极小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 59.36275101139466
agent 2 在 5999 次迭代中的资源申请量为: 59.162577596028356
agent 3 在 5999 次迭代中的资源申请量为: 64.10362875798097
agent 4 在 5999 次迭代中的资源申请量为: 40.0767960026118
agent 5 在 5999 次迭代中的资源申请量为: 60.79915796992848
agent 6 在 5999 次迭代中的资源申请量为: 58.438336014138855
agent 7 在 5999 次迭代中的资源申请量为: 69.40119892410384
agent 8 在 5999 次迭代中的资源申请量为: 48.92915368421673
agent 9 在 5999 次迭代中的资源申请量为: 69.36777569634499
agent 10 在 5999 次迭代中的资源申请量为: 57.286743795837815
agent 11 在 5999 次迭代中的资源申请量为: 69.26151830772274
agent 12 在 5999 次迭代中的资源申请量为: 40.57404746142212
agent 13 在 5999 次迭代中的资源申请量为: 52.55990720375692
agent 14 在 5999 次迭代中的资源申请量为: 43.65843447470364
agent 15 在 5999 次迭代中的资源申请量为: 44.86411348641598
agent 16 在 5999 次迭代中的资源申请量为: 52.015899296965074
agent 17 在 5999 次迭代中的资源申请量为: 52.909124268084405
agent 18 在 5999 次迭代中的资源申请量为: 49.15263390148715
agent 19 在 5999 次迭代中的资源申请量为: 47.336518595354605
agent 20 在 5999 次迭代中的资源申请量为: 53.62381754820506
agent 21 在 5999 次迭代中的资源申请量为: 73.89651594758332
age

8999 次迭代中的结果：
agent 1 在 8999 次迭代中的资源申请量为: 59.36757623222927
agent 2 在 8999 次迭代中的资源申请量为: 59.168030934974475
agent 3 在 8999 次迭代中的资源申请量为: 64.10936318684917
agent 4 在 8999 次迭代中的资源申请量为: 40.080021016139334
agent 5 在 8999 次迭代中的资源申请量为: 60.80417629130517
agent 6 在 8999 次迭代中的资源申请量为: 58.443300574992996
agent 7 在 8999 次迭代中的资源申请量为: 69.40704142174444
agent 8 在 8999 次迭代中的资源申请量为: 48.93318593884379
agent 9 在 8999 次迭代中的资源申请量为: 69.3738302612764
agent 10 在 8999 次迭代中的资源申请量为: 57.29203898908649
agent 11 在 8999 次迭代中的资源申请量为: 69.26718920533253
agent 12 在 8999 次迭代中的资源申请量为: 40.576823451402
agent 13 在 8999 次迭代中的资源申请量为: 52.5643927835371
agent 14 在 8999 次迭代中的资源申请量为: 43.66223709173635
agent 15 在 8999 次迭代中的资源申请量为: 44.86758646684807
agent 16 在 8999 次迭代中的资源申请量为: 52.01989173452773
agent 17 在 8999 次迭代中的资源申请量为: 52.913385636573
agent 18 在 8999 次迭代中的资源申请量为: 49.15674318787188
agent 19 在 8999 次迭代中的资源申请量为: 47.3403074684513
agent 20 在 8999 次迭代中的资源申请量为: 53.62839690380876
agent 21 在 8999 次迭代中的资源申请量为: 73.90278422355259
agent 22 在 8

11999 次迭代中的结果：
agent 1 在 11999 次迭代中的资源申请量为: 59.367668258342825
agent 2 在 11999 次迭代中的资源申请量为: 59.1685780477684
agent 3 在 11999 次迭代中的资源申请量为: 64.10977105210907
agent 4 在 11999 次迭代中的资源申请量为: 40.080054311693516
agent 5 在 11999 次迭代中的资源申请量为: 60.804329237011814
agent 6 在 11999 次迭代中的资源申请量为: 58.44355125849565
agent 7 在 11999 次迭代中的资源申请量为: 69.40720520738186
agent 8 在 11999 次迭代中的资源申请量为: 48.9333077434788
agent 9 在 11999 次迭代中的资源申请量为: 69.37418593504565
agent 10 在 11999 次迭代中的资源申请量为: 57.29255622526451
agent 11 在 11999 次迭代中的资源申请量为: 69.2672081404575
agent 12 在 11999 次迭代中的资源申请量为: 40.57653562312412
agent 13 在 11999 次迭代中的资源申请量为: 52.5645586474657
agent 14 在 11999 次迭代中的资源申请量为: 43.662506684673325
agent 15 在 11999 次迭代中的资源申请量为: 44.867556323728984
agent 16 在 11999 次迭代中的资源申请量为: 52.01981812997338
agent 17 在 11999 次迭代中的资源申请量为: 52.913470457294885
agent 18 在 11999 次迭代中的资源申请量为: 49.156899170027074
agent 19 在 11999 次迭代中的资源申请量为: 47.340303065719354
agent 20 在 11999 次迭代中的资源申请量为: 53.62866297701618
agent 21 在 11999 次迭代中的资源申请量为: 

14999 次迭代中的结果：
agent 1 在 14999 次迭代中的资源申请量为: 59.36768109038193
agent 2 在 14999 次迭代中的资源申请量为: 59.16893540469499
agent 3 在 14999 次迭代中的资源申请量为: 64.11001727216086
agent 4 在 14999 次迭代中的资源申请量为: 40.08004106057919
agent 5 在 14999 次迭代中的资源申请量为: 60.80438687272048
agent 6 在 14999 次迭代中的资源申请量为: 58.4436851646455
agent 7 在 14999 次迭代中的资源申请量为: 69.40726126110113
agent 8 在 14999 次迭代中的资源申请量为: 48.933353209669725
agent 9 在 14999 次迭代中的资源申请量为: 69.37438789774565
agent 10 在 14999 次迭代中的资源申请量为: 57.29289237911856
agent 11 在 14999 次迭代中的资源申请量为: 69.2671541927769
agent 12 在 14999 次迭代中的资源申请量为: 40.57627933387612
agent 13 在 14999 次迭代中的资源申请量为: 52.56463272175274
agent 14 在 14999 次迭代中的资源申请量为: 43.662669277863884
agent 15 在 14999 次迭代中的资源申请量为: 44.86749098806227
agent 16 在 14999 次迭代中的资源申请量为: 52.01971287227491
agent 17 在 14999 次迭代中的资源申请量为: 52.913484533969545
agent 18 在 14999 次迭代中的资源申请量为: 49.15697015147299
agent 19 在 14999 次迭代中的资源申请量为: 47.340253823985464
agent 20 在 14999 次迭代中的资源申请量为: 53.6288134546728
agent 21 在 14999 次迭代中的资源申请量为: 73.

...

47999 次迭代中的结果：
agent 1 在 47999 次迭代中的资源申请量为: 59.3677397050285
agent 2 在 47999 次迭代中的资源申请量为: 59.170670679948174
agent 3 在 47999 次迭代中的资源申请量为: 64.1112114054673
agent 4 在 47999 次迭代中的资源申请量为: 40.07997399831246
agent 5 在 47999 次迭代中的资源申请量为: 60.804663489787636
agent 6 在 47999 次迭代中的资源申请量为: 58.44433304532953
agent 7 在 47999 次迭代中的资源申请量为: 69.40752953581054
agent 8 在 47999 次迭代中的资源申请量为: 48.93357132276618
agent 9 在 47999 次迭代中的资源申请量为: 69.37536630897243
agent 10 在 47999 次迭代中的资源申请量为: 57.29452457853478
agent 11 在 47999 次迭代中的资源申请量为: 69.26688712425072
agent 12 在 47999 次迭代中的资源申请量为: 40.57502949451721
agent 13 在 47999 次迭代中的资源申请量为: 52.56498976141653
agent 14 在 47999 次迭代中的资源申请量为: 43.66345772252609
agent 15 在 47999 次迭代中的资源申请量为: 44.86717019147399
agent 16 在 47999 次迭代中的资源申请量为: 52.01919717045603
agent 17 在 47999 次迭代中的资源申请量为: 52.91354963866336
agent 18 在 47999 次迭代中的资源申请量为: 49.15731245055273
agent 19 在 47999 次迭代中的资源申请量为: 47.34001109983776
agent 20 在 47999 次迭代中的资源申请量为: 53.62954236825567
agent 21 在 47999 次迭代中的资源申请量为: 73.9

50999 次迭代中的结果：
agent 1 在 50999 次迭代中的资源申请量为: 59.367742660779626
agent 2 在 50999 次迭代中的资源申请量为: 59.170755798329914
agent 3 在 50999 次迭代中的资源申请量为: 64.11127001912374
agent 4 在 50999 次迭代中的资源申请量为: 40.07997076942454
agent 5 在 50999 次迭代中的资源申请量为: 60.804677132825695
agent 6 在 50999 次迭代中的资源申请量为: 58.444364875234754
agent 7 在 50999 次迭代中的资源申请量为: 69.40754278522026
agent 8 在 50999 次迭代中的资源申请量为: 48.93358207976177
agent 9 在 50999 次迭代中的资源申请量为: 69.37541435746468
agent 10 在 50999 次迭代中的资源申请量为: 57.294604644524014
agent 11 在 50999 次迭代中的资源申请量为: 69.26687414084059
agent 12 在 50999 次迭代中的资源申请量为: 40.574968304125285
agent 13 在 50999 次迭代中的资源申请量为: 52.565007334458684
agent 14 在 50999 次迭代中的资源申请量为: 43.66349641882822
agent 15 在 50999 次迭代中的资源申请量为: 44.86715453523334
agent 16 在 50999 次迭代中的资源申请量为: 52.01917196638974
agent 17 在 50999 次迭代中的资源申请量为: 52.9135529017732
agent 18 在 50999 次迭代中的资源申请量为: 49.15732929552996
agent 19 在 50999 次迭代中的资源申请量为: 47.33999927334167
agent 20 在 50999 次迭代中的资源申请量为: 53.629578164395
agent 21 在 50999 次迭代中的资源申请量为: 

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 59.36774543594748
agent 2 在 53999 次迭代中的资源申请量为: 59.170835584273355
agent 3 在 53999 次迭代中的资源申请量为: 64.11132496318909
agent 4 在 53999 次迭代中的资源申请量为: 40.079967746303176
agent 5 在 53999 次迭代中的资源申请量为: 60.80468992548402
agent 6 在 53999 次迭代中的资源申请量为: 58.44439471392179
agent 7 在 53999 次迭代中的资源申请量为: 69.4075552098243
agent 8 在 53999 次迭代中的资源申请量为: 48.93359216617054
agent 9 在 53999 次迭代中的资源申请量为: 69.37545939914665
agent 10 在 53999 次迭代中的资源申请量为: 57.29467969483093
agent 11 在 53999 次迭代中的资源申请量为: 69.26686197757616
agent 12 在 53999 次迭代中的资源申请量为: 40.57491095377198
agent 13 在 53999 次迭代中的资源申请量为: 52.56502381003105
agent 14 在 53999 次迭代中的资源申请量为: 43.663532692103864
agent 15 在 53999 次迭代中的资源申请量为: 44.867139864387525
agent 16 在 53999 次迭代中的资源申请量为: 52.019148346348686
agent 17 在 53999 次迭代中的资源申请量为: 52.913555964393964
agent 18 在 53999 次迭代中的资源申请量为: 49.15734508838744
agent 19 在 53999 次迭代中的资源申请量为: 47.339988192286896
agent 20 在 53999 次迭代中的资源申请量为: 53.62961172043614
agent 21 在 53999 次迭代中的资源申请量为

56999 次迭代中的结果：
agent 1 在 56999 次迭代中的资源申请量为: 59.367748050494434
agent 2 在 56999 次迭代中的资源申请量为: 59.17091064135651
agent 3 在 56999 次迭代中的资源申请量为: 64.11137665279466
agent 4 在 56999 次迭代中的资源申请量为: 40.079964905298965
agent 5 在 56999 次迭代中的资源申请量为: 60.804701963577244
agent 6 在 56999 次迭代中的资源申请量为: 58.44442278647849
agent 7 在 56999 次迭代中的资源申请量为: 69.40756690245071
agent 8 在 56999 次迭代中的资源申请量为: 48.9336016575514
agent 9 在 56999 次迭代中的资源申请量为: 69.3755017740316
agent 10 在 56999 次迭代中的资源申请量为: 57.29475029715899
agent 11 在 56999 次迭代中的资源申请量为: 69.26685054093554
agent 12 在 56999 次迭代中的资源申请量为: 40.574857008113234
agent 13 在 56999 次迭代中的资源申请量为: 52.56503931200365
agent 14 在 56999 次迭代中的资源申请量为: 43.66356681648579
agent 15 在 56999 次迭代中的资源申请量为: 44.86712606693023
agent 16 在 56999 次迭代中的资源申请量为: 52.019126130487805
agent 17 在 56999 次迭代中的资源申请量为: 52.91355884880584
agent 18 在 56999 次迭代中的资源申请量为: 49.15735994789228
agent 19 在 56999 次迭代中的资源申请量为: 47.33997777182573
agent 20 在 56999 次迭代中的资源申请量为: 53.62964328969022
agent 21 在 56999 次迭代中的资源申请量为: 7

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 59.367750521288464
agent 2 在 59999 次迭代中的资源申请量为: 59.17098147744166
agent 3 在 59999 次迭代中的资源申请量为: 64.1114254372729
agent 4 在 59999 次迭代中的资源申请量为: 40.07996222656601
agent 5 在 59999 次迭代中的资源申请量为: 60.804713327790594
agent 6 在 59999 次迭代中的资源申请量为: 58.44444928234269
agent 7 在 59999 次迭代中的资源申请量为: 69.40757794128537
agent 8 在 59999 次迭代中的资源申请量为: 48.93361061753242
agent 9 在 59999 次迭代中的资源申请量为: 69.37554176826121
agent 10 在 59999 次迭代中的资源申请量为: 57.2948169291877
agent 11 在 59999 次迭代中的资源申请量为: 69.2668397523295
agent 12 在 59999 次迭代中的资源申请量为: 40.57480610094498
agent 13 在 59999 次迭代中的资源申请量为: 52.56505394465536
agent 14 在 59999 次迭代中的资源申请量为: 43.663599022654225
agent 15 在 59999 次迭代中的资源申请量为: 44.867113048686726
agent 16 在 59999 次迭代中的资源申请量为: 52.01910516756518
agent 17 在 59999 次迭代中的资源申请量为: 52.913561573817525
agent 18 在 59999 次迭代中的资源申请量为: 49.15737397402491
agent 19 在 59999 次迭代中的资源申请量为: 47.33996794063575
agent 20 在 59999 次迭代中的资源申请量为: 53.62967308533423
agent 21 在 59999 次迭代中的资源申请量为: 73

In [6]:
np.savetxt('./data/03_SCC_oracle_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_oracle_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_oracle_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_oracle_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
tau_robust2=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust2=[]
neighbor_byzantine_num_robust2=[]
for i in range(0,94):
    h_num_robust2=0
    b_num_robust2=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            b_num_robust2=b_num_robust2+1
        else:
            h_num_robust2=h_num_robust2+1
    neighbor_honest_num_robust2.append(h_num_robust2)
    neighbor_byzantine_num_robust2.append(b_num_robust2)

In [10]:
#DGD+CC算法 
def Robust_DGD_SCC_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,tau_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust2*100)):
            c_clipping_dual_robust2=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust2=0#存放clipping后的对偶值和
            #计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust2[i]==0:
                tau_robust2[i][k]=10000
            else:
                weight_sum_robust2=0
                for j in range(0,15):
                    if neighbor_robust2[i][j]!=94 and neighbor_robust2[i][j]!=95 and neighbor_robust2[i][j]!=96 and neighbor_robust2[i][j]!=97 and neighbor_robust2[i][j]!=98 and neighbor_robust2[i][j]!=99:
                        weight_sum_robust2=weight_sum_robust2+(0.0625)*(dual_variable_intermediate_robust2[neighbor_robust2[i][j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i][j]]-dual_variable_intermediate_robust2[i])
                tau_robust2[i][k]=sqrt(weight_sum_robust2/(0.0625*neighbor_byzantine_num_robust2[i]))                      
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])==0:
                    c_clipping_dual_robust2.append(0)
                elif 1<=(tau_robust2[i][k]/(math.sqrt((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])))):
                    c_clipping_dual_robust2.append(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])
                else:
                    c_clipping_dual_robust2.append((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*((tau_robust2[i][k]/(math.sqrt((dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i])*(dual_variable_intermediate_robust2[neighbor_robust2[i,j]]-dual_variable_intermediate_robust2[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust2=mix_sum_c_dual_robust2+0.0625*(dual_variable_intermediate_robust2[i]+c_clipping_dual_robust2[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust2[i][k]=0.0625*dual_variable_intermediate_robust2[i]+mix_sum_c_dual_robust2
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_SCC_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,tau_robust2)

****************************DGD算法+SCC+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 53.30522323121256
agent 2 在 3999 次迭代中的资源申请量为: 53.16857462644728
agent 3 在 3999 次迭代中的资源申请量为: 57.48806451220256
agent 4 在 3999 次迭代中的资源申请量为: 35.971973345417034
agent 5 在 3999 次迭代中的资源申请量为: 54.61064904994471
agent 6 在 3999 次迭代中的资源申请量为: 52.50884392327649
agent 7 在 3999 次迭代中的资源申请量为: 62.16561949472848
agent 8 在 3999 次迭代中的资源申请量为: 43.96633052681893
agent 9 在 3999 次迭代中的资源申请量为: 62.24056225532525
agent 10 在 3999 次迭代中的资源申请量为: 51.4193280981392
agent 11 在 3999 次迭代中的资源申请量为: 61.94934296295172
agent 12 在 3999 次迭代中的资源申请量为: 36.44331069400429
agent 13 在 3999 次迭代中的资源申请量为: 47.0970495160418
agent 14 在 3999 次迭代中的资源申请量为: 39.26927221862095
agent 15 在 3999 次迭代中的资源申请量为: 40.327104116859516
agent 16 在 3999 次迭代中的资源申请量为: 46.72651858071353
agent 17 在 3999 次迭代中的资源申请量为: 47.56950778167254
agent 18 在 3999 次迭代中的资源申请量为: 44.157393019947634
agent 19 在 3999 次迭代中的资源申请量为: 42.45127790201884
agent 20 在 3999 次迭代中的资源申请量为: 48.2195311773677
agent 21 在 3999 次迭代中的资源申请量为: 66.29639922179503
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 53.33921874610858
agent 2 在 5999 次迭代中的资源申请量为: 53.20223689804476
agent 3 在 5999 次迭代中的资源申请量为: 57.52489296311965
agent 4 在 5999 次迭代中的资源申请量为: 35.99507146559489
agent 5 在 5999 次迭代中的资源申请量为: 54.64533401214055
agent 6 在 5999 次迭代中的资源申请量为: 52.54202881755967
agent 7 在 5999 次迭代中的资源申请量为: 62.20628591289344
agent 8 在 5999 次迭代中的资源申请量为: 43.99375242598739
agent 9 在 5999 次迭代中的资源申请量为: 62.2801773766273
agent 10 在 5999 次迭代中的资源申请量为: 51.45295309089805
agent 11 在 5999 次迭代中的资源申请量为: 61.99108614873749
agent 12 在 5999 次迭代中的资源申请量为: 36.466155879678006
agent 13 在 5999 次迭代中的资源申请量为: 47.127248425870825
agent 14 在 5999 次迭代中的资源申请量为: 39.29372373207508
agent 15 在 5999 次迭代中的资源申请量为: 40.3522841451475
agent 16 在 5999 次迭代中的资源申请量为: 46.75584102895377
agent 17 在 5999 次迭代中的资源申请量为: 47.59936578474123
agent 18 在 5999 次迭代中的资源申请量为: 44.18495350507631
agent 19 在 5999 次迭代中的资源申请量为: 42.47806478346506
agent 20 在 5999 次迭代中的资源申请量为: 48.24984092454152
agent 21 在 5999 次迭代中的资源申请量为: 66.33936478152859
agent 22

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 53.340644123679986
agent 2 在 7999 次迭代中的资源申请量为: 53.20359426108512
agent 3 在 7999 次迭代中的资源申请量为: 57.526160473658464
agent 4 在 7999 次迭代中的资源申请量为: 35.996091756014344
agent 5 在 7999 次迭代中的资源申请量为: 54.646744254725995
agent 6 在 7999 次迭代中的资源申请量为: 52.5433246626565
agent 7 在 7999 次迭代中的资源申请量为: 62.20802864008072
agent 8 在 7999 次迭代中的资源申请量为: 43.994566674106224
agent 9 在 7999 次迭代中的资源申请量为: 62.281506781747375
agent 10 在 7999 次迭代中的资源申请量为: 51.454828456610066
agent 11 在 7999 次迭代中的资源申请量为: 61.993393171661765
agent 12 在 7999 次迭代中的资源申请量为: 36.466911946153715
agent 13 在 7999 次迭代中的资源申请量为: 47.12815073794799
agent 14 在 7999 次迭代中的资源申请量为: 39.294579047571006
agent 15 在 7999 次迭代中的资源申请量为: 40.35314007811949
agent 16 在 7999 次迭代中的资源申请量为: 46.75681805459732
agent 17 在 7999 次迭代中的资源申请量为: 47.60053461275437
agent 18 在 7999 次迭代中的资源申请量为: 44.18573485658078
agent 19 在 7999 次迭代中的资源申请量为: 42.478719400473125
agent 20 在 7999 次迭代中的资源申请量为: 48.25106756222374
agent 21 在 7999 次迭代中的资源申请量为: 66.3413816540054

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 53.34074817716754
agent 2 在 9999 次迭代中的资源申请量为: 53.203654033079395
agent 3 在 9999 次迭代中的资源申请量为: 57.52605173238125
agent 4 在 9999 次迭代中的资源申请量为: 35.99620394939772
agent 5 在 9999 次迭代中的资源申请量为: 54.646815161455244
agent 6 在 9999 次迭代中的资源申请量为: 52.54334974186669
agent 7 在 9999 次迭代中的资源申请量为: 62.208183255640954
agent 8 在 9999 次迭代中的资源申请量为: 43.99438201030007
agent 9 在 9999 次迭代中的资源申请量为: 62.28136257801531
agent 10 在 9999 次迭代中的资源申请量为: 51.455304083199565
agent 11 在 9999 次迭代中的资源申请量为: 61.99396672979314
agent 12 在 9999 次迭代中的资源申请量为: 36.46682032301907
agent 13 在 9999 次迭代中的资源申请量为: 47.12795186323906
agent 14 在 9999 次迭代中的资源申请量为: 39.294517874809586
agent 15 在 9999 次迭代中的资源申请量为: 40.353057177170655
agent 16 在 9999 次迭代中的资源申请量为: 46.75670572454794
agent 17 在 9999 次迭代中的资源申请量为: 47.6005595056424
agent 18 在 9999 次迭代中的资源申请量为: 44.18551963089364
agent 19 在 9999 次迭代中的资源申请量为: 42.47842631202067
agent 20 在 9999 次迭代中的资源申请量为: 48.25112494771689
agent 21 在 9999 次迭代中的资源申请量为: 66.34168561047187
age

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 53.34104384299815
agent 2 在 51999 次迭代中的资源申请量为: 53.203654473354526
agent 3 在 51999 次迭代中的资源申请量为: 57.524875397895094
agent 4 在 51999 次迭代中的资源申请量为: 35.996685509358215
agent 5 在 51999 次迭代中的资源申请量为: 54.64687858753121
agent 6 在 51999 次迭代中的资源申请量为: 52.54312122017453
agent 7 在 51999 次迭代中的资源申请量为: 62.208740774764216
agent 8 在 51999 次迭代中的资源申请量为: 43.992805019309124
agent 9 在 51999 次迭代中的资源申请量为: 62.27991303738636
agent 10 在 51999 次迭代中的资源申请量为: 51.45811647821801
agent 11 在 51999 次迭代中的资源申请量为: 61.99734411805557
agent 12 在 51999 次迭代中的资源申请量为: 36.46592719388311
agent 13 在 51999 次迭代中的资源申请量为: 47.12624559014151
agent 14 在 51999 次迭代中的资源申请量为: 39.29381121468474
agent 15 在 51999 次迭代中的资源申请量为: 40.35219494629702
agent 16 在 51999 次迭代中的资源申请量为: 46.75559475890112
agent 17 在 51999 次迭代中的资源申请量为: 47.600369945690204
agent 18 在 51999 次迭代中的资源申请量为: 44.18373444355468
agent 19 在 51999 次迭代中的资源申请量为: 42.4761240442211
agent 20 在 51999 次迭代中的资源申请量为: 48.25114935207762
agent 21 在 51999 次迭代中的资源申请量为:

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 53.34105005945897
agent 2 在 53999 次迭代中的资源申请量为: 53.203654486335004
agent 3 在 53999 次迭代中的资源申请量为: 57.52485068130289
agent 4 在 53999 次迭代中的资源申请量为: 35.996695636495474
agent 5 在 53999 次迭代中的资源申请量为: 54.6468799242988
agent 6 在 53999 次迭代中的资源申请量为: 52.54311641659551
agent 7 在 53999 次迭代中的资源申请量为: 62.20875249697331
agent 8 在 53999 次迭代中的资源申请量为: 43.99277187490343
agent 9 在 53999 次迭代中的资源申请量为: 62.27988257667231
agent 10 在 53999 次迭代中的资源申请量为: 51.458175608486776
agent 11 在 53999 次迭代中的资源申请量为: 61.997415132333
agent 12 在 53999 次迭代中的资源申请量为: 36.46590842520413
agent 13 在 53999 次迭代中的资源申请量为: 47.12620973012095
agent 14 在 53999 次迭代中的资源申请量为: 39.29379636361658
agent 15 在 53999 次迭代中的资源申请量为: 40.35217682597816
agent 16 在 53999 次迭代中的资源申请量为: 46.755571407712246
agent 17 在 53999 次迭代中的资源申请量为: 47.600365967755366
agent 18 在 53999 次迭代中的资源申请量为: 44.183696923961094
agent 19 在 53999 次迭代中的资源申请量为: 42.47607565490509
agent 20 在 53999 次迭代中的资源申请量为: 48.25114986629311
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 53.34105602903778
agent 2 在 55999 次迭代中的资源申请量为: 53.20365450020843
agent 3 在 55999 次迭代中的资源申请量为: 57.52482695340853
agent 4 在 55999 次迭代中的资源申请量为: 35.996705360322125
agent 5 在 55999 次迭代中的资源申请量为: 54.64688120913523
agent 6 在 55999 次迭代中的资源申请量为: 52.543111806055094
agent 7 在 55999 次迭代中的资源申请量为: 62.20876375277038
agent 8 在 55999 次迭代中的资源申请量为: 43.9927400551067
agent 9 在 55999 次迭代中的资源申请量为: 62.27985333406397
agent 10 在 55999 次迭代中的资源申请量为: 51.45823237956195
agent 11 在 55999 次迭代中的资源申请量为: 61.997483313520156
agent 12 在 55999 次迭代中的资源申请量为: 36.465890406994404
agent 13 在 55999 次迭代中的资源申请量为: 47.12617530332404
agent 14 在 55999 次迭代中的资源申请量为: 39.29378210657164
agent 15 在 55999 次迭代中的资源申请量为: 40.35215943028174
agent 16 在 55999 次迭代中的资源申请量为: 46.75554899006232
agent 17 在 55999 次迭代中的资源申请量为: 47.600362150043395
agent 18 在 55999 次迭代中的资源申请量为: 44.18366090372366
agent 19 在 55999 次迭代中的资源申请量为: 42.47602919890628
agent 20 在 55999 次迭代中的资源申请量为: 48.25115036115923
agent 21 在 55999 次迭代中的资源申请量为: 

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 53.341061769798365
agent 2 在 57999 次迭代中的资源申请量为: 53.20365451480696
agent 3 在 57999 次迭代中的资源申请量为: 57.52480414135078
agent 4 在 57999 次迭代中的资源申请量为: 35.99671471043665
agent 5 在 57999 次迭代中的资源申请量为: 54.64688244575636
agent 6 在 57999 次迭代中的资源申请量为: 52.54310737425649
agent 7 在 57999 次迭代中的资源申请量为: 62.208774576348794
agent 8 在 57999 次迭代中的资源申请量为: 43.99270946238088
agent 9 在 57999 次迭代中的资源申请量为: 62.27982521981652
agent 10 在 57999 次迭代中的资源申请量为: 51.45828696489462
agent 11 在 57999 次迭代中的资源申请量为: 61.99754886989768
agent 12 在 57999 次迭代中的资源申请量为: 36.46587308396772
agent 13 在 57999 次迭代中的资源申请量为: 47.126142204210744
agent 14 在 57999 次迭代中的资源申请量为: 39.29376839977219
agent 15 在 57999 次迭代中的资源申请量为: 40.352142705818245
agent 16 在 57999 次迭代中的资源申请量为: 46.755527437183694
agent 17 在 57999 次迭代中的资源申请量为: 47.60035848067047
agent 18 在 57999 次迭代中的资源申请量为: 44.18362627244575
agent 19 在 57999 次迭代中的资源申请量为: 42.47598453390421
agent 20 在 57999 次迭代中的资源申请量为: 48.25115083801098
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 53.34106729791898
agent 2 在 59999 次迭代中的资源申请量为: 53.20365452999054
agent 3 在 59999 次迭代中的资源申请量为: 57.52478217992651
agent 4 在 59999 次迭代中的资源申请量为: 35.99672371333873
agent 5 在 59999 次迭代中的资源申请量为: 54.646883637498505
agent 6 在 59999 次迭代中的资源申请量为: 52.54310310841278
agent 7 在 59999 次迭代中的资源申请量为: 62.20878499832578
agent 8 在 59999 次迭代中的资源申请量为: 43.992680009432114
agent 9 在 59999 次迭代中的资源申请量为: 62.2797981536155
agent 10 在 59999 次迭代中的资源申请量为: 51.45833951974492
agent 11 在 59999 次迭代中的资源申请量为: 61.997611987903305
agent 12 在 59999 次迭代中的资源申请量为: 36.46585640664685
agent 13 在 59999 次迭代中的资源申请量为: 47.126110338326114
agent 14 在 59999 次迭代中的资源申请量为: 39.29375520404207
agent 15 在 59999 次迭代中的资源申请量为: 40.352126604808696
agent 16 在 59999 次迭代中的资源申请量为: 46.75550668753395
agent 17 在 59999 次迭代中的资源申请量为: 47.60035494900935
agent 18 在 59999 次迭代中的资源申请量为: 44.183592931324526
agent 19 在 59999 次迭代中的资源申请量为: 42.475941532522064
agent 20 在 59999 次迭代中的资源申请量为: 48.251151298052186
agent 21 在 59999 次迭代中的资源申请量

In [12]:
np.savetxt('./data/03_SCC_oracle_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_oracle_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_oracle_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_oracle_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust3_help=[]
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
tau_robust3=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust3=[]
neighbor_byzantine_num_robust3=[]
for i in range(0,94):
    h_num_robust3=0
    b_num_robust3=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            b_num_robust3=b_num_robust3+1
        else:
            h_num_robust3=h_num_robust3+1
    neighbor_honest_num_robust3.append(h_num_robust3)
    neighbor_byzantine_num_robust3.append(b_num_robust3)

In [16]:
def Robust_DGD_SCC_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,tau_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust3*100)):
            c_clipping_dual_robust3=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust3=0#存放clipping后的对偶值和
            #计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust3[i]==0:
                tau_robust3[i][k]=10000
            else:
                weight_sum_robust3=0
                for j in range(0,15):
                    if neighbor_robust3[i][j]!=94 and neighbor_robust3[i][j]!=95 and neighbor_robust3[i][j]!=96 and neighbor_robust3[i][j]!=97 and neighbor_robust3[i][j]!=98 and neighbor_robust3[i][j]!=99:
                        weight_sum_robust3=weight_sum_robust3+(0.0625)*(dual_variable_intermediate_robust3[neighbor_robust3[i][j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i][j]]-dual_variable_intermediate_robust3[i])
                tau_robust3[i][k]=sqrt(weight_sum_robust3/(0.0625*neighbor_byzantine_num_robust3[i]))            
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])==0:
                    c_clipping_dual_robust3.append(0)
                elif 1<=(tau_robust3[i][k]/(math.sqrt((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])))):
                    c_clipping_dual_robust3.append(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])
                else:
                    c_clipping_dual_robust3.append((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*((tau_robust3[i][k]/(math.sqrt((dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i])*(dual_variable_intermediate_robust3[neighbor_robust3[i,j]]-dual_variable_intermediate_robust3[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust3=mix_sum_c_dual_robust3+0.0625*(dual_variable_intermediate_robust3[i]+c_clipping_dual_robust3[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust3[i][k]=0.0625*dual_variable_intermediate_robust3[i]+mix_sum_c_dual_robust3
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_SCC_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,tau_robust3)

****************************DGD算法+SCC+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 53.30538786710946
agent 2 在 3999 次迭代中的资源申请量为: 53.16873805529683
agent 3 在 3999 次迭代中的资源申请量为: 57.48824495005496
agent 4 在 3999 次迭代中的资源申请量为: 35.97208481666797
agent 5 在 3999 次迭代中的资源申请量为: 54.61081735627188
agent 6 在 3999 次迭代中的资源申请量为: 52.509005352568614
agent 7 在 3999 次迭代中的资源申请量为: 62.16581615364746
agent 8 在 3999 次迭代中的资源申请量为: 43.96646585291188
agent 9 在 3999 次迭代中的资源申请量为: 62.24075660220523
agent 10 在 3999 次迭代中的资源申请量为: 51.419487436010726
agent 11 在 3999 次迭代中的资源申请量为: 61.94954092940499
agent 12 在 3999 次迭代中的资源申请量为: 36.44342284925612
agent 13 在 3999 次迭代中的资源申请量为: 47.09719850440613
agent 14 在 3999 次迭代中的资源申请量为: 39.26939191297089
agent 15 在 3999 次迭代中的资源申请量为: 40.32722756459065
agent 16 在 3999 次迭代中的资源申请量为: 46.726662485449715
agent 17 在 3999 次迭代中的资源申请量为: 47.569653005497344
agent 18 在 3999 次迭代中的资源申请量为: 44.15752930855038
agent 19 在 3999 次迭代中的资源申请量为: 42.45141115390744
agent 20 在 3999 次迭代中的资源申请量为: 48.219678296261385
agent 21 在 3999 次迭代中的资源申请量为: 66.29660567730775
age

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 53.33922542901313
agent 2 在 5999 次迭代中的资源申请量为: 53.20224353194175
agent 3 在 5999 次迭代中的资源申请量为: 57.524900287401714
agent 4 在 5999 次迭代中的资源申请量为: 35.995075990452236
agent 5 在 5999 次迭代中的资源申请量为: 54.64534084402744
agent 6 在 5999 次迭代中的资源申请量为: 52.542035370283415
agent 7 在 5999 次迭代中的资源申请量为: 62.20629389569153
agent 8 在 5999 次迭代中的资源申请量为: 43.99375791908286
agent 9 在 5999 次迭代中的资源申请量为: 62.280185265497764
agent 10 在 5999 次迭代中的资源申请量为: 51.45295955884667
agent 11 在 5999 次迭代中的资源申请量为: 61.99109418473319
agent 12 在 5999 次迭代中的资源申请量为: 36.46616043224998
agent 13 在 5999 次迭代中的资源申请量为: 47.127254473536745
agent 14 在 5999 次迭代中的资源申请量为: 39.29372859067969
agent 15 在 5999 次迭代中的资源申请量为: 40.352289156101904
agent 16 在 5999 次迭代中的资源申请量为: 46.75584687028713
agent 17 在 5999 次迭代中的资源申请量为: 47.59937167965719
agent 18 在 5999 次迭代中的资源申请量为: 44.18495903723127
agent 19 在 5999 次迭代中的资源申请量为: 42.47807019233553
agent 20 在 5999 次迭代中的资源申请量为: 48.249846896384916
agent 21 在 5999 次迭代中的资源申请量为: 66.33937316201762
a

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 53.340644425661644
agent 2 在 7999 次迭代中的资源申请量为: 53.20359456085194
agent 3 在 7999 次迭代中的资源申请量为: 57.52616080462046
agent 4 在 7999 次迭代中的资源申请量为: 35.996091960480335
agent 5 在 7999 次迭代中的资源申请量为: 54.6467445634395
agent 6 在 7999 次迭代中的资源申请量为: 52.54332495875511
agent 7 在 7999 次迭代中的资源申请量为: 62.20802900080137
agent 8 在 7999 次迭代中的资源申请量为: 43.994566922321766
agent 9 在 7999 次迭代中的资源申请量为: 62.28150713822129
agent 10 在 7999 次迭代中的资源申请量为: 51.45482874888163
agent 11 在 7999 次迭代中的资源申请量为: 61.99339353479004
agent 12 在 7999 次迭代中的资源申请量为: 36.4669121518705
agent 13 在 7999 次迭代中的资源申请量为: 47.12815101122272
agent 14 在 7999 次迭代中的资源申请量为: 39.29457926711672
agent 15 在 7999 次迭代中的资源申请量为: 40.35314030454922
agent 16 在 7999 次迭代中的资源申请量为: 46.75681831854916
agent 17 在 7999 次迭代中的资源申请量为: 47.60053487912864
agent 18 在 7999 次迭代中的资源申请量为: 44.185735106560976
agent 19 在 7999 次迭代中的资源申请量为: 42.47871964488188
agent 20 在 7999 次迭代中的资源申请量为: 48.25106783207424
agent 21 在 7999 次迭代中的资源申请量为: 66.34138203269752
agent 

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 53.34074819191099
agent 2 在 9999 次迭代中的资源申请量为: 53.2036540477147
agent 3 在 9999 次迭代中的资源申请量为: 57.52605174853951
agent 4 在 9999 次迭代中的资源申请量为: 35.996203959380225
agent 5 在 9999 次迭代中的资源申请量为: 54.64681517652732
agent 6 在 9999 次迭代中的资源申请量为: 52.54334975632287
agent 7 在 9999 次迭代中的资源申请量为: 62.20818327325215
agent 8 在 9999 次迭代中的资源申请量为: 43.99438202241844
agent 9 在 9999 次迭代中的资源申请量为: 62.28136259541911
agent 10 在 9999 次迭代中的资源申请量为: 51.455304097469046
agent 11 在 9999 次迭代中的资源申请量为: 61.99396674752202
agent 12 在 9999 次迭代中的资源申请量为: 36.466820333062586
agent 13 在 9999 次迭代中的资源申请量为: 47.12795187658088
agent 14 在 9999 次迭代中的资源申请量为: 39.29451788552828
agent 15 在 9999 次迭代中的资源申请量为: 40.353057188225414
agent 16 在 9999 次迭代中的资源申请量为: 46.756705737434615
agent 17 在 9999 次迭代中的资源申请量为: 47.60055951864739
agent 18 在 9999 次迭代中的资源申请量为: 44.18551964309815
agent 19 在 9999 次迭代中的资源申请量为: 42.47842632395315
agent 20 在 9999 次迭代中的资源申请量为: 48.251124960891595
agent 21 在 9999 次迭代中的资源申请量为: 66.34168562896055
age

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 53.34104384299815
agent 2 在 51999 次迭代中的资源申请量为: 53.203654473354526
agent 3 在 51999 次迭代中的资源申请量为: 57.524875397895094
agent 4 在 51999 次迭代中的资源申请量为: 35.996685509358215
agent 5 在 51999 次迭代中的资源申请量为: 54.64687858753121
agent 6 在 51999 次迭代中的资源申请量为: 52.54312122017453
agent 7 在 51999 次迭代中的资源申请量为: 62.208740774764216
agent 8 在 51999 次迭代中的资源申请量为: 43.992805019309124
agent 9 在 51999 次迭代中的资源申请量为: 62.27991303738636
agent 10 在 51999 次迭代中的资源申请量为: 51.45811647821801
agent 11 在 51999 次迭代中的资源申请量为: 61.99734411805557
agent 12 在 51999 次迭代中的资源申请量为: 36.46592719388311
agent 13 在 51999 次迭代中的资源申请量为: 47.12624559014151
agent 14 在 51999 次迭代中的资源申请量为: 39.29381121468474
agent 15 在 51999 次迭代中的资源申请量为: 40.35219494629702
agent 16 在 51999 次迭代中的资源申请量为: 46.75559475890112
agent 17 在 51999 次迭代中的资源申请量为: 47.600369945690204
agent 18 在 51999 次迭代中的资源申请量为: 44.18373444355468
agent 19 在 51999 次迭代中的资源申请量为: 42.4761240442211
agent 20 在 51999 次迭代中的资源申请量为: 48.25114935207762
agent 21 在 51999 次迭代中的资源申请量为:

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 53.34105005945897
agent 2 在 53999 次迭代中的资源申请量为: 53.203654486335004
agent 3 在 53999 次迭代中的资源申请量为: 57.52485068130289
agent 4 在 53999 次迭代中的资源申请量为: 35.996695636495474
agent 5 在 53999 次迭代中的资源申请量为: 54.6468799242988
agent 6 在 53999 次迭代中的资源申请量为: 52.54311641659551
agent 7 在 53999 次迭代中的资源申请量为: 62.20875249697331
agent 8 在 53999 次迭代中的资源申请量为: 43.99277187490343
agent 9 在 53999 次迭代中的资源申请量为: 62.27988257667231
agent 10 在 53999 次迭代中的资源申请量为: 51.458175608486776
agent 11 在 53999 次迭代中的资源申请量为: 61.997415132333
agent 12 在 53999 次迭代中的资源申请量为: 36.46590842520413
agent 13 在 53999 次迭代中的资源申请量为: 47.12620973012095
agent 14 在 53999 次迭代中的资源申请量为: 39.29379636361658
agent 15 在 53999 次迭代中的资源申请量为: 40.35217682597816
agent 16 在 53999 次迭代中的资源申请量为: 46.755571407712246
agent 17 在 53999 次迭代中的资源申请量为: 47.600365967755366
agent 18 在 53999 次迭代中的资源申请量为: 44.183696923961094
agent 19 在 53999 次迭代中的资源申请量为: 42.47607565490509
agent 20 在 53999 次迭代中的资源申请量为: 48.25114986629311
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 53.34105602903778
agent 2 在 55999 次迭代中的资源申请量为: 53.20365450020843
agent 3 在 55999 次迭代中的资源申请量为: 57.52482695340853
agent 4 在 55999 次迭代中的资源申请量为: 35.996705360322125
agent 5 在 55999 次迭代中的资源申请量为: 54.64688120913523
agent 6 在 55999 次迭代中的资源申请量为: 52.543111806055094
agent 7 在 55999 次迭代中的资源申请量为: 62.20876375277038
agent 8 在 55999 次迭代中的资源申请量为: 43.9927400551067
agent 9 在 55999 次迭代中的资源申请量为: 62.27985333406397
agent 10 在 55999 次迭代中的资源申请量为: 51.45823237956195
agent 11 在 55999 次迭代中的资源申请量为: 61.997483313520156
agent 12 在 55999 次迭代中的资源申请量为: 36.465890406994404
agent 13 在 55999 次迭代中的资源申请量为: 47.12617530332404
agent 14 在 55999 次迭代中的资源申请量为: 39.29378210657164
agent 15 在 55999 次迭代中的资源申请量为: 40.35215943028174
agent 16 在 55999 次迭代中的资源申请量为: 46.75554899006232
agent 17 在 55999 次迭代中的资源申请量为: 47.600362150043395
agent 18 在 55999 次迭代中的资源申请量为: 44.18366090372366
agent 19 在 55999 次迭代中的资源申请量为: 42.47602919890628
agent 20 在 55999 次迭代中的资源申请量为: 48.25115036115923
agent 21 在 55999 次迭代中的资源申请量为: 

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 53.341061769798365
agent 2 在 57999 次迭代中的资源申请量为: 53.20365451480696
agent 3 在 57999 次迭代中的资源申请量为: 57.52480414135078
agent 4 在 57999 次迭代中的资源申请量为: 35.99671471043665
agent 5 在 57999 次迭代中的资源申请量为: 54.64688244575636
agent 6 在 57999 次迭代中的资源申请量为: 52.54310737425649
agent 7 在 57999 次迭代中的资源申请量为: 62.208774576348794
agent 8 在 57999 次迭代中的资源申请量为: 43.99270946238088
agent 9 在 57999 次迭代中的资源申请量为: 62.27982521981652
agent 10 在 57999 次迭代中的资源申请量为: 51.45828696489462
agent 11 在 57999 次迭代中的资源申请量为: 61.99754886989768
agent 12 在 57999 次迭代中的资源申请量为: 36.46587308396772
agent 13 在 57999 次迭代中的资源申请量为: 47.126142204210744
agent 14 在 57999 次迭代中的资源申请量为: 39.29376839977219
agent 15 在 57999 次迭代中的资源申请量为: 40.352142705818245
agent 16 在 57999 次迭代中的资源申请量为: 46.755527437183694
agent 17 在 57999 次迭代中的资源申请量为: 47.60035848067047
agent 18 在 57999 次迭代中的资源申请量为: 44.18362627244575
agent 19 在 57999 次迭代中的资源申请量为: 42.47598453390421
agent 20 在 57999 次迭代中的资源申请量为: 48.25115083801098
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 53.34106729791898
agent 2 在 59999 次迭代中的资源申请量为: 53.20365452999054
agent 3 在 59999 次迭代中的资源申请量为: 57.52478217992651
agent 4 在 59999 次迭代中的资源申请量为: 35.99672371333873
agent 5 在 59999 次迭代中的资源申请量为: 54.646883637498505
agent 6 在 59999 次迭代中的资源申请量为: 52.54310310841278
agent 7 在 59999 次迭代中的资源申请量为: 62.20878499832578
agent 8 在 59999 次迭代中的资源申请量为: 43.992680009432114
agent 9 在 59999 次迭代中的资源申请量为: 62.2797981536155
agent 10 在 59999 次迭代中的资源申请量为: 51.45833951974492
agent 11 在 59999 次迭代中的资源申请量为: 61.997611987903305
agent 12 在 59999 次迭代中的资源申请量为: 36.46585640664685
agent 13 在 59999 次迭代中的资源申请量为: 47.126110338326114
agent 14 在 59999 次迭代中的资源申请量为: 39.29375520404207
agent 15 在 59999 次迭代中的资源申请量为: 40.352126604808696
agent 16 在 59999 次迭代中的资源申请量为: 46.75550668753395
agent 17 在 59999 次迭代中的资源申请量为: 47.60035494900935
agent 18 在 59999 次迭代中的资源申请量为: 44.183592931324526
agent 19 在 59999 次迭代中的资源申请量为: 42.475941532522064
agent 20 在 59999 次迭代中的资源申请量为: 48.251151298052186
agent 21 在 59999 次迭代中的资源申请量

In [18]:
np.savetxt('./data/03_SCC_oracle_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_oracle_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_oracle_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_oracle_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+SCC算法+高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust4_help=[]
random_attack3_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
tau_robust4=np.zeros(shape=(100,60000))#每个用户在每次迭代中均需要计算出一个tau值
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack3_robust4[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

#计算出94个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust4=[]
neighbor_byzantine_num_robust4=[]
for i in range(0,94):
    h_num_robust4=0
    b_num_robust4=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            b_num_robust4=b_num_robust4+1
        else:
            h_num_robust4=h_num_robust4+1
    neighbor_honest_num_robust4.append(h_num_robust4)
    neighbor_byzantine_num_robust4.append(b_num_robust4)

In [22]:
def Robust_DGD_SCC_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,tau_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+SCC+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])              
        #阶段二：SCC鲁棒聚合 每个用户聚合自身以及邻居的对偶值
        for i in range (0,100-int(proportion_robust4*100)):
            c_clipping_dual_robust4=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust4=0#存放clipping后的对偶值和
            #计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust4[i]==0:
                tau_robust4[i][k]=10000
            else:
                weight_sum_robust4=0
                for j in range(0,15):
                    if neighbor_robust4[i][j]!=94 and neighbor_robust4[i][j]!=95 and neighbor_robust4[i][j]!=96 and neighbor_robust4[i][j]!=97 and neighbor_robust4[i][j]!=98 and neighbor_robust4[i][j]!=99:
                        weight_sum_robust4=weight_sum_robust4+(0.0625)*(dual_variable_intermediate_robust4[neighbor_robust4[i][j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i][j]]-dual_variable_intermediate_robust4[i])
                tau_robust4[i][k]=sqrt(weight_sum_robust4/(0.0625*neighbor_byzantine_num_robust4[i]))            
            #每个诚实用户首先对15个邻居传来的对偶变量进行剪裁clipping
            for j in range (0, 15): #问题：分母会出现0的情况
                if (dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])==0:
                    c_clipping_dual_robust4.append(0)
                elif 1<=(tau_robust4[i][k]/(math.sqrt((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])))):
                    c_clipping_dual_robust4.append(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])
                else:
                    c_clipping_dual_robust4.append((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*((tau_robust4[i][k]/(math.sqrt((dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i])*(dual_variable_intermediate_robust4[neighbor_robust4[i,j]]-dual_variable_intermediate_robust4[i]))))))    
            #诚实用户聚合clipping后的15个邻居对偶值和自身的对偶值 按照权重聚合 Wij=0.0625
            for j in range(0,15):
                mix_sum_c_dual_robust4=mix_sum_c_dual_robust4+0.0625*(dual_variable_intermediate_robust4[i]+c_clipping_dual_robust4[j])
            #诚实用户基于CC鲁棒聚合后的值进行自身的对偶变量的更新
            dual_variable_robust4[i][k]=0.0625*dual_variable_intermediate_robust4[i]+mix_sum_c_dual_robust4
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack3_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_SCC_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,tau_robust4)

****************************DGD算法+SCC+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 59.28201836102185
agent 2 在 3999 次迭代中的资源申请量为: 59.081771716242976
agent 3 在 3999 次迭代中的资源申请量为: 64.01471381354929
agent 4 在 3999 次迭代中的资源申请量为: 40.022172588334364
agent 5 在 3999 次迭代中的资源申请量为: 60.716542453925925
agent 6 在 3999 次迭代中的资源申请量为: 58.35894619523069
agent 7 在 3999 次迭代中的资源申请量为: 69.30467877600326
agent 8 在 3999 次迭代中的资源申请量为: 48.862746000850436
agent 9 在 3999 次迭代中的资源申请量为: 69.27213151027533
agent 10 在 3999 次迭代中的资源申请量为: 57.20795131433531
agent 11 在 3999 次迭代中的资源申请量为: 69.16453484757167
agent 12 在 3999 次迭代中的资源申请量为: 40.51957541423798
agent 13 在 3999 次迭代中的资源申请量为: 52.48674876586718
agent 14 在 3999 次迭代中的资源申请量为: 43.59945291487834
agent 15 在 3999 次迭代中的资源申请量为: 44.80373570793025
agent 16 在 3999 次迭代中的资源申请量为: 51.945573651141
agent 17 在 3999 次迭代中的资源申请量为: 52.83790927225807
agent 18 在 3999 次迭代中的资源申请量为: 49.0857076271123
agent 19 在 3999 次迭代中的资源申请量为: 47.27131986766379
agent 20 在 3999 次迭代中的资源申请量为: 53.55140808436107
agent 21 在 3999 次迭代中的资源申请量为: 73.79504564337363
agent 2

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 59.361106194464305
agent 2 在 5999 次迭代中的资源申请量为: 59.160945377059036
agent 3 在 5999 次迭代中的资源申请量为: 64.10182672991301
agent 4 在 5999 次迭代中的资源申请量为: 40.075682229031784
agent 5 在 5999 次迭代中的资源申请量为: 60.79747661259583
agent 6 在 5999 次迭代中的资源申请量为: 58.43672351580524
agent 7 在 5999 次迭代中的资源申请量为: 69.39923412338308
agent 8 在 5999 次迭代中的资源申请量为: 48.92780214764765
agent 9 在 5999 次迭代中的资源申请量为: 69.36583471617796
agent 10 在 5999 次迭代中的资源申请量为: 57.285151742273555
agent 11 在 5999 次迭代中的资源申请量为: 69.25953956620485
agent 12 在 5999 次迭代中的资源申请量为: 40.57292683307
agent 13 在 5999 次迭代中的资源申请量为: 52.55841925583299
agent 14 在 5999 次迭代中的资源申请量为: 43.657239071454576
agent 15 在 5999 次迭代中的资源申请量为: 44.86288030721877
agent 16 在 5999 次迭代中的资源申请量为: 52.01446173968854
agent 17 在 5999 次迭代中的资源申请量为: 52.90767347327177
agent 18 在 5999 次迭代中的资源申请量为: 49.15127283723275
agent 19 在 5999 次迭代中的资源申请量为: 47.33518782175561
agent 20 在 5999 次迭代中的资源申请量为: 53.6223479895007
agent 21 在 5999 次迭代中的资源申请量为: 73.89445322828946
agent 2

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 59.36701254673959
agent 2 在 7999 次迭代中的资源申请量为: 59.167281136965364
agent 3 在 7999 次迭代中的资源申请量为: 64.1086173440107
agent 4 在 7999 次迭代中的资源申请量为: 40.07965144562043
agent 5 在 7999 次迭代中的资源申请量为: 60.803575485812644
agent 6 在 7999 次迭代中的资源申请量为: 58.442680921959706
agent 7 在 7999 次迭代中的资源申请量为: 69.40634557717156
agent 8 在 7999 次迭代中的资源申请量为: 48.93270346419921
agent 9 在 7999 次迭代中的资源申请量为: 69.37306185147591
agent 10 在 7999 次迭代中的资源申请量为: 57.291314499561324
agent 11 在 7999 次迭代中的资源申请量为: 69.26654942730094
agent 12 在 7999 次迭代中的资源申请量为: 40.576585857576084
agent 13 在 7999 次迭代中的资源申请量为: 52.563848341513335
agent 14 在 7999 次迭代中的资源申请量为: 43.66174272852529
agent 15 在 7999 次迭代中的资源申请量为: 44.8672052721212
agent 16 在 7999 次迭代中的资源申请量为: 52.01946343312473
agent 17 在 7999 次迭代中的资源申请量为: 52.912886917728486
agent 18 在 7999 次迭代中的资源申请量为: 49.156243393096275
agent 19 在 7999 次迭代中的资源申请量为: 47.33988435469329
agent 20 在 7999 次迭代中的资源申请量为: 53.62781646676615
agent 21 在 7999 次迭代中的资源申请量为: 73.90201515689606
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 59.36761763344348
agent 2 在 9999 次迭代中的资源申请量为: 59.16824047105505
agent 3 在 9999 次迭代中的资源申请量为: 64.10952202154326
agent 4 在 9999 次迭代中的资源申请量为: 40.08003832551857
agent 5 在 9999 次迭代中的资源申请量为: 60.804240370914194
agent 6 在 9999 次迭代中的资源申请量为: 58.44340046805615
agent 7 在 9999 次迭代中的资源申请量为: 69.4071107809105
agent 8 在 9999 次迭代中的资源申请量为: 48.93323702793035
agent 9 在 9999 次迭代中的资源申请量为: 69.37397043150114
agent 10 在 9999 次迭代中的资源申请量为: 57.29223730274554
agent 11 在 9999 次迭代中的资源申请量为: 69.26720509296332
agent 12 在 9999 次迭代中的资源申请量为: 40.57672211029294
agent 13 在 9999 次迭代中的资源申请量为: 52.564460768799485
agent 14 在 9999 次迭代中的资源申请量为: 43.662342099495206
agent 15 在 9999 次迭代中的资源申请量为: 44.867580866808765
agent 16 在 9999 次迭代中的资源申请量为: 52.01987099178413
agent 17 在 9999 次迭代中的资源申请量为: 52.91342350332543
agent 18 在 9999 次迭代中的资源申请量为: 49.156806950764405
agent 19 在 9999 次迭代中的资源申请量为: 47.34031182023885
agent 20 在 9999 次迭代中的资源申请量为: 53.62850184177852
agent 21 在 9999 次迭代中的资源申请量为: 73.9028911352825
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 59.3665200244959
agent 2 在 51999 次迭代中的资源申请量为: 59.16956867779629
agent 3 在 51999 次迭代中的资源申请量为: 64.10994814405474
agent 4 在 51999 次迭代中的资源申请量为: 40.079141205800354
agent 5 在 51999 次迭代中的资源申请量为: 60.80343071116818
agent 6 在 51999 次迭代中的资源申请量为: 58.44317546794359
agent 7 在 51999 次迭代中的资源申请量为: 69.40608539244947
agent 8 在 51999 次迭代中的资源申请量为: 48.93258009453234
agent 9 在 51999 次迭代中的资源申请量为: 69.37398574966646
agent 10 在 51999 次迭代中的资源申请量为: 57.29344579238309
agent 11 在 51999 次迭代中的资源申请量为: 69.26539802520955
agent 12 在 51999 次迭代中的资源申请量为: 40.57411519790249
agent 13 在 51999 次迭代中的资源申请量为: 52.56390604342997
agent 14 在 51999 次迭代中的资源申请量为: 43.66261947164464
agent 15 在 51999 次迭代中的资源申请量为: 44.866232192884354
agent 16 在 51999 次迭代中的资源申请量为: 52.018094556903876
agent 17 在 51999 次迭代中的资源申请量为: 52.91247469645899
agent 18 在 51999 次迭代中的资源申请量为: 49.15632216322464
agent 19 在 51999 次迭代中的资源申请量为: 47.33900555039358
agent 20 在 51999 次迭代中的资源申请量为: 53.62849635189511
agent 21 在 51999 次迭代中的资源申请量为: 73

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 59.366865122029026
agent 2 在 53999 次迭代中的资源申请量为: 59.169961963441374
agent 3 在 53999 次迭代中的资源申请量为: 64.1103604757565
agent 4 在 53999 次迭代中的资源申请量为: 40.079371651211005
agent 5 在 53999 次迭代中的资源申请量为: 60.80379004583979
agent 6 在 53999 次迭代中的资源申请量为: 58.443531679309295
agent 7 在 53999 次迭代中的资源申请量为: 69.40650363572293
agent 8 在 53999 次迭代中的资源申请量为: 48.93286881076943
agent 9 在 53999 次迭代中的资源申请量为: 69.37442054702191
agent 10 在 53999 次迭代中的资源申请量为: 57.29382757156593
agent 11 在 53999 次迭代中的资源申请量为: 69.26580295440135
agent 12 在 53999 次迭代中的资源申请量为: 40.57431122484161
agent 13 在 53999 次迭代中的资源申请量为: 52.564227443767116
agent 14 在 53999 次迭代中的资源申请量为: 43.66289288335063
agent 15 在 53999 次迭代中的资源申请量为: 44.86647987137923
agent 16 在 53999 次迭代中的资源申请量为: 52.01837898070612
agent 17 在 53999 次迭代中的资源申请量为: 52.91277949196168
agent 18 在 53999 次迭代中的资源申请量为: 49.15661663247875
agent 19 在 53999 次迭代中的资源申请量为: 47.33927596490205
agent 20 在 53999 次迭代中的资源申请量为: 53.628825185354714
agent 21 在 53999 次迭代中的资源申请量为:

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 59.367654506213746
agent 2 在 55999 次迭代中的资源申请量为: 59.17079412980369
agent 3 在 55999 次迭代中的资源申请量为: 64.1112582094586
agent 4 在 55999 次迭代中的资源申请量为: 40.07990306965621
agent 5 在 55999 次迭代中的资源申请量为: 60.804603279834105
agent 6 在 55999 次迭代中的资源申请量为: 58.44432274267598
agent 7 在 55999 次迭代中的资源申请量为: 69.40745235939701
agent 8 在 55999 次迭代中的资源申请量为: 48.93352239259157
agent 9 在 55999 次迭代中的资源申请量为: 69.37537854446497
agent 10 在 55999 次迭代中的资源申请量为: 57.29463750479309
agent 11 在 55999 次迭代中的资源申请量为: 69.26674277148179
agent 12 在 55999 次迭代中的资源申请量为: 40.57481148929997
agent 13 在 55999 次迭代中的资源申请量为: 52.56495039550522
agent 14 在 55999 次迭代中的资源申请量为: 43.66348829901168
agent 15 在 55999 次迭代中的资源申请量为: 44.867061083582875
agent 16 在 55999 次迭代中的资源申请量为: 52.019052382026
agent 17 在 55999 次迭代中的资源申请量为: 52.913476150016464
agent 18 在 55999 次迭代中的资源申请量为: 49.15727839173723
agent 19 在 55999 次迭代中的资源申请量为: 47.33990618513251
agent 20 在 55999 次迭代中的资源申请量为: 53.629550158035244
agent 21 在 55999 次迭代中的资源申请量为: 73

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 59.36773905120081
agent 2 在 57999 次迭代中的资源申请量为: 59.17092493634423
agent 3 在 57999 次迭代中的资源申请量为: 64.1113824426381
agent 4 在 57999 次迭代中的资源申请量为: 40.07995733347612
agent 5 在 57999 次迭代中的资源申请量为: 60.80469576614235
agent 6 在 57999 次迭代中的资源申请量为: 58.444422140141064
agent 7 在 57999 次迭代中的资源申请量为: 69.4075588991483
agent 8 在 57999 次迭代中的资源申请量为: 48.93359661641744
agent 9 在 57999 次迭代中的资源申请量为: 69.37550374726884
agent 10 在 57999 次迭代中的资源申请量为: 57.29476340453938
agent 11 在 57999 次迭代中的资源申请量为: 69.2668350409398
agent 12 在 57999 次迭代中的资源申请量为: 40.57483301545722
agent 13 在 57999 次迭代中的资源申请量为: 52.565035381541264
agent 14 在 57999 次迭代中的资源申请量为: 43.66357060438497
agent 15 在 57999 次迭代中的资源申请量为: 44.86711426927883
agent 16 在 57999 次迭代中的资源申请量为: 52.01911041221482
agent 17 在 57999 次迭代中的资源申请量为: 52.91355109640278
agent 18 在 57999 次迭代中的资源申请量为: 49.1573565703996
agent 19 在 57999 次迭代中的资源申请量为: 47.33996647285266
agent 20 在 57999 次迭代中的资源申请量为: 53.62964461903135
agent 21 在 57999 次迭代中的资源申请量为: 73.903

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 59.367749468967226
agent 2 在 59999 次迭代中的资源申请量为: 59.17098043311876
agent 3 在 59999 次迭代中的资源申请量为: 64.11142428432953
agent 4 在 59999 次迭代中的资源申请量为: 40.07996151399814
agent 5 在 59999 次迭代中的资源申请量为: 60.804712252080186
agent 6 在 59999 次迭代中的资源申请量为: 58.444448250676096
agent 7 在 59999 次迭代中的资源申请量为: 69.40757668424067
agent 8 在 59999 次迭代中的资源申请量为: 48.93360975283803
agent 9 在 59999 次迭代中的资源申请量为: 69.375540526423
agent 10 在 59999 次迭代中的资源申请量为: 57.29481591056288
agent 11 在 59999 次迭代中的资源申请量为: 69.26683848638092
agent 12 在 59999 次迭代中的资源申请量为: 40.57480538403504
agent 13 在 59999 次迭代中的资源申请量为: 52.5650529926841
agent 14 在 59999 次迭代中的资源申请量为: 43.66359825782999
agent 15 在 59999 次迭代中的资源申请量为: 44.86711225973612
agent 16 在 59999 次迭代中的资源申请量为: 52.01910424787165
agent 17 在 59999 次迭代中的资源申请量为: 52.91356064562769
agent 18 在 59999 次迭代中的资源申请量为: 49.157373103233276
agent 19 在 59999 次迭代中的资源申请量为: 47.33996708924444
agent 20 在 59999 次迭代中的资源申请量为: 53.62967214511459
agent 21 在 59999 次迭代中的资源申请量为: 73.

In [24]:
np.savetxt('./data/03_SCC_oracle_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/03_SCC_oracle_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/03_SCC_oracle_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/03_SCC_oracle_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量